In [1]:
import pandas as pd
dataset = pd.read_csv('amazon_baby.csv')
dataset.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


In [2]:
dataset.info()
dataset.isnull().sum()
#Handling missing data
dataset=dataset.fillna({'review':''})


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183531 entries, 0 to 183530
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   name    183213 non-null  object
 1   review  182702 non-null  object
 2   rating  183531 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 4.2+ MB


In [3]:
dataset.isnull().sum()

name      318
review      0
rating      0
dtype: int64

In [5]:
def remove_punctuation(text):
    import string
    table=text.maketrans('','',string.punctuation)
    return text.translate(table)

In [6]:
dataset['review']=dataset['review'].astype('str')

In [7]:
dataset['review_clean']=dataset['review'].apply(remove_punctuation)

In [8]:
dataset.isnull().sum()

name            318
review            0
rating            0
review_clean      0
dtype: int64

In [9]:
#consider only rows having rating is not 3
dataset=dataset[dataset['rating']!=3]

In [10]:
dataset['sentiment']=dataset['rating'].apply(lambda rating:1 if rating>3 else -1)

In [11]:
with open('train_id.json','r') as f:
    id_train = f.readline()
with open('test_id.json','r') as f:
    id_test=f.readline()

import numpy as np
id_train=list(map(int,id_train[1:-2].split(',')))
id_test=list(map(int,id_test[1:-2].split(',')))

In [13]:
train_data=dataset.iloc[id_train,:]
test_data=dataset.iloc[id_test,:]

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer(token_pattern=r'\b\w+\b')
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
test_matrix=vectorizer.transform(test_data['review_clean'])

In [15]:
train_matrix

<133416x121710 sparse matrix of type '<class 'numpy.int64'>'
	with 7326546 stored elements in Compressed Sparse Row format>

In [16]:
from sklearn.linear_model import LogisticRegression


In [17]:
sentiment_model = LogisticRegression()
sentiment_model.fit(train_matrix,train_data['sentiment'])

C:\Users\Soumia\anaconda3\envs\project\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [18]:
sample_test_data=test_data[10:13]


In [19]:
sample_test_data['review']

59    Absolutely love it and all of the Scripture in...
71    Would not purchase again or recommend. The dec...
91    Was so excited to get this product for my baby...
Name: review, dtype: object

In [20]:
sample_test_data.iloc[1,1]


'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

In [21]:
sample_test_matrix=vectorizer.transform(sample_test_data.iloc[:,1])

In [22]:
prediction = sentiment_model.predict(sample_test_matrix)

In [23]:
prediction

array([ 1, -1, -1], dtype=int64)

In [24]:
import pickle

In [25]:
file1=open("vectorizer.pkl",'wb')
pickle.dump(vectorizer,file1)
file1.close()
file = open('model.pkl','wb')
pickle.dump(sentiment_model,file)
file.close()

In [26]:
t = vectorizer.transform(["I will not recommend it. It's not so good."])